# Importing Masterliste Werke+Aufnahmen

The sheet "machine readable" contains the fixed data including manually added folder and file names for csv data files (tappings)

## Preparation
Download sheet "machine readable" as csv.


In [15]:
%matplotlib inline

# Adding data from notebooks

## Audio conversion

In [3]:
from pydub import AudioSegment

data_dir = "/home/martin/Workspace/musicData/data/recordings/BRUCKNER_Sym9-3_-_ABBADO_WPh_1996_LIVE_DG_[471 032-2]"
file_name = data_dir + "/" + "03_BRUCKNER_Sym9-3_-_ABBADO_WPh_1996_LIVE_DG_[471 032-2].flac"

song = AudioSegment.from_file(file_name, "flac")
song.export(file_name + ".wav", format="wav")



<_io.BufferedRandom name='/home/martin/Workspace/musicData/data/recordings/BRUCKNER_Sym9-3_-_ABBADO_WPh_1996_LIVE_DG_[471 032-2]/03_BRUCKNER_Sym9-3_-_ABBADO_WPh_1996_LIVE_DG_[471 032-2].flac.wav'>

## OMA API

<Response [200]>


## Using the OMA API to upload the data

In [19]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import math
import os
from pydub import AudioSegment
from omapy.api import oma
import librosa


url = "http://localhost:8080/"
login = oma.login(url, "oma", "=PSe?sZ-ymp6mE>2", False)
print(login)

data_list = "data/Masterliste Werke+Aufnahmen - machine readable.csv"
data_dir = "/home/martin/Workspace/musicData/data/"
df = pd.read_csv(data_list)

df.head()



filtered_data = df[df['tapping_files'].notnull()]
    
for index, row in filtered_data.iterrows():
    
    tapping_file_list = row['tapping_files'].split(",")
    
    # parse row
    composer = row["Komponist"]
    work = row["Werkname"]
    part_number = row["Satz_Nr"]
    conductor = row["Dirigent"]
    orchestra = row["Orchester"]
    year = row["Aufnahmejahr"]
    
    # print(composer, work, part_number, conductor, orchestra, year)
    
    c = {"name": composer}
    c = oma.composer_add(c)
    print(c)

    composition = {"title": work, "composer": c["id"]}
    composition = oma.composition_add(composition)
    print(composition)
    
    # interpretation
    interpretation = {"title":  conductor + ", " + orchestra + ", " + str(year)}
    interpretation = oma.interpretation_add(interpretation)
    print(interpretation)
    
    
    # abstract music (work) part of composition
    
    if math.isnan(row["Takt von"]):
        row["Takt von"] = 0.0
    
    if math.isnan(row["Takt bis"]):
        row["Takt bis"] = 0.0
    
    # offset and lengths in OMA instead of start and end. It's equivalent   
    row["Takt bis"] = row["Takt bis"] - row["Takt von"] 

    
    abstract_music_part = {
        "title": "Movement " + str(part_number),
        "movement": part_number,
        "composition": composition["id"],
        "interpretation": interpretation["id"],
        "bar_number_offset": row["Takt von"],
        "number_of_bars": row["Takt bis"]
    }
    abstract_music_part = oma.interpretation_add_abstract_music_part(abstract_music_part)
    print(abstract_music_part)
    
    
    # recording and audio files
    audio_path = data_dir + "recordings/" +row["recording_folder"]
    audio_file_list = os.listdir(audio_path)
    legit_audio_files = []
    for a in audio_file_list:
        if a.endswith(".flac"): # or .wav or whatever
            legit_audio_files.append(a)
    
    if (len(legit_audio_files) != 1):
        print ("only single audio file supported")
        sys.exit(1)
    
    file_name = legit_audio_files[0]
    audio_file = audio_path + file_name
    wav_file = audio_file + ".wav"
    mp3_file = audio_file + ".64k.mp3"
    
    sound = AudioSegment.from_file(audio_file, "flac")
    # song.export(wav_file, format="wav")
    
    
    if os.path.isfile(mp3_file) == False:
        print("converting")
        sound.export(mp3_file, format="mp3", bitrate="64k")
        
    
    recording = {
        "title": "Recording of " + str(row["Aufnahmejahr"]),
        "interpretation": interpretation["id"],
        "abstract_music_part": abstract_music_part["id"]       
    }
    files = {
        # "file": (wav_file, open(wav_file, 'rb'), 'audio/wav', {'Expires': '0'})
        "file": (mp3_file, open(mp3_file, 'rb'), 'audio/mpeg', {'Expires': '0'})
    
    }
    
    recording = oma.recording_add(recording, files)
    print(recording)
    
    
    mono_sound = sound.set_channels(1)
    
    sr = mono_sound.frame_rate
    bytes_per_frame = mono_sound.frame_width
    
    sound_samples = np.array(mono_sound.get_array_of_samples())
    
    print(sr)
    print(bytes_per_frame)
    print(sound_samples)
    
    
    
    #sns.set()
    
    # librosa.display.waveplot(sound_samples, sr=sr) maybe?
    
    
    break
    
    
    
    
    # tappings. multiple files possible
    for i in range(0, len(tapping_file_list)):
        # print("\t", tapping_file_list[i])
        # import that tapping file to recording
        cwd = data_dir + row["tapping_folder"]
        tapping_file_name = tapping_file_list[i].rstrip()
        tapping_file_path = cwd + "/" + tapping_file_name
        # print(tapping_file_path)
        if os.path.isfile(tapping_file_path) == False:
            print("File not found")
        # tapping_data = parse_t_csv(cwd + "/" + tapping_file_name)
        # upload tapping data
        session = {
            "recording": recording["id"],
            "tapping_file_name": tapping_file_name
        }
        files = {
            "file": (tapping_file_name, open(tapping_file_path, 'rb'), 'text/csv', {'Expires': '0'})
    
        }
        # session = oma.session_add(session, files)
        # print(session["id"])
        
    print("")
    print("Done!")
        



<Response [200]>
{'tenantId': 3, 'compositions': [{'id': 6}], 'name': 'Anton Bruckner', 'id': 5}
{'tenantId': 3, 'title': 'Symphonie No. 9', 'composer': {'tenantId': 3, 'compositions': [{'_ref': '../..', 'class': 'org.karajanresearch.oma.music.AbstractMusic'}], 'name': 'Anton Bruckner', 'id': 5}, 'id': 6}
{'tenantId': 3, 'abstractMusicParts': [{'id': 8}], 'title': 'Eugen Jochum, Berliner Philharmoniker, 1966', 'recordings': [{'id': 9}], 'id': 7}
{'tenantId': 3, 'interpretationOrder': 3, 'interpretation': {'tenantId': 3, 'abstractMusicParts': [{'_ref': '../..', 'class': 'org.karajanresearch.oma.music.AbstractMusicPart'}], 'title': 'Eugen Jochum, Berliner Philharmoniker, 1966', 'recordings': [{'tenantId': 3, 'abstractMusicPart': {'_ref': '../../..', 'class': 'org.karajanresearch.oma.music.AbstractMusicPart'}, 'recordingData': {}, 'digitalAudio': [{'tenantId': 3, 'location': 'https://s3.amazonaws.com/open-music-annotations-storage-backend/test/recording/9/2020-01-13-12-07-815_03_BRUCKNER_

AttributeError: module 'librosa' has no attribute 'display'